<a href="https://colab.research.google.com/github/fabriziobasso/kaggle/blob/main/File_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<h1 align="center"><font color='#DD4141'> S4E9 Regression of Used Car Prices**</font></h1>

In [163]:
%%capture
# Connect to Colab:#
from google.colab import drive
import os
drive.mount('/content/drive')

!pip install category-encoders
!pip install optuna
!pip install optuna-integration
!pip install scikit-learn==1.4
!pip install catboost

In [164]:
folder_script = models_folders = "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E9_Cars"
os.chdir(folder_script)

In [165]:
# Setup feedback system
#from learntools.core import binder
#binder.bind(globals())
#from learntools.time_series.ex6 import *

# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import numpy as np
from pickle import load, dump
import json
import joblib
#import calplot as cal

# Graphic Libraries:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
# Set Style
sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5});
sns.despine(left=True, bottom=True, top=False, right=False);
mpl.rcParams['figure.dpi'] = 120;
mpl.rc('axes', labelsize=12);
plt.rc('xtick',labelsize=10);
plt.rc('ytick',labelsize=10);

mpl.rcParams['axes.spines.top'] = False;
mpl.rcParams['axes.spines.right'] = False;
mpl.rcParams['axes.spines.left'] = True;

# Palette Setup
colors = ['#FB5B68','#FFEB48','#2676A1','#FFBDB0',]
colormap_0 = mpl.colors.LinearSegmentedColormap.from_list("",colors)
palette_1 = sns.color_palette("coolwarm", as_cmap=True)
palette_2 = sns.color_palette("YlOrBr", as_cmap=True)
palette_3 = sns.light_palette("red", as_cmap=True)
palette_4 = sns.color_palette("viridis", as_cmap=True)
palette_5 = sns.color_palette("rocket", as_cmap=True)
palette_6 = sns.color_palette("GnBu", as_cmap=True)
palette_7 = sns.color_palette("tab20c", as_cmap=False)
palette_8 = sns.color_palette("Set2", as_cmap=False)

palette_custom = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palette_9 = sns.color_palette(palette_custom, as_cmap=False)

# tool for Excel:
from openpyxl import load_workbook, Workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.worksheet.cell_range import CellRange

from openpyxl.formatting import Rule
from openpyxl.styles import Font, PatternFill, Border
from openpyxl.styles.differential import DifferentialStyle

# Bloomberg
#from xbbg import blp
from catboost import CatBoostRegressor, Pool
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
import lightgbm as lgb
from lightgbm import (LGBMRegressor,
                      LGBMClassifier,
                      early_stopping,
                      record_evaluation,
                      log_evaluation)

# Time Management
from tqdm import tqdm
from datetime import date
from datetime import datetime
from pandas.tseries.offsets import BMonthEnd, QuarterEnd
import datetime
from pandas.tseries.offsets import BDay # BDay is business day, not birthday...
import datetime as dt
import click
import glob
import os
import gc
import re
import string

from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output

#from my_func import *

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

from keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.losses import MeanSquaredError

#from my_func import *

# preprocessing modules
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_validate, GroupKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_predict

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   OrdinalEncoder,
                                   RobustScaler,
                                   PowerTransformer,
                                   OneHotEncoder,
                                   LabelEncoder,
                                   QuantileTransformer,
                                   PolynomialFeatures)

# metrics
import sklearn
from sklearn.metrics import (mean_squared_error,
                             root_mean_squared_error,
                             r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             multilabel_confusion_matrix,
                             accuracy_score,
                             roc_auc_score,
                             auc,
                             roc_curve,
                             log_loss,
                             make_scorer)


# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso,
                                  ridge_regression,
                                  LinearRegression,
                                  Ridge,
                                  RidgeCV,
                                  ElasticNet,
                                  BayesianRidge,
                                  HuberRegressor,
                                  TweedieRegressor,
                                  ARDRegression,
                                  PoissonRegressor,
                                  GammaRegressor)


from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

import itertools
import warnings
from openpyxl import load_workbook

import statsmodels.api as sm
from pylab import rcParams
import scipy.stats as ss

warnings.filterwarnings('ignore')
#plt.style.use('fivethirtyeight')

<Figure size 960x660 with 0 Axes>

In [166]:
import xgboost
print(xgboost.__version__)
print(sklearn.__version__)

2.1.1
1.4.0


In [167]:
%%time
#import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
import os
import seaborn as sns
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import logging
import warnings

# Setting rc parameters in seaborn for plots and graphs-
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.75,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
#sns.set_theme(style="ticks", context="notebook")
sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

CPU times: user 1.63 ms, sys: 0 ns, total: 1.63 ms
Wall time: 1.62 ms


<Figure size 960x660 with 0 Axes>

In [168]:
def plot_categorical_columns(df):
    # Select only categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns

    # Loop through each categorical column and plot
    for col in categorical_columns:
        if len(set(df[col]))<150:
            plt.figure(figsize=(6, 4))
            sns.countplot(y=df[col], palette=palette_9, order=df[col].value_counts().index)
            plt.title(f'Distribution of {col}')
            plt.xlabel('Count')
            plt.ylabel(col)
            plt.tight_layout()
            plt.show()

In [169]:
#plot_categorical_columns(df_train)

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: white; font-size:100%; text-align:left">

**<h3 align="center"><font color='#3455eb'> Used Cars Regression :</font></h3>**

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: white; font-size:100%; text-align:left">

<h3 align="center"><font color='#3455eb'> About The Competition :</font></h3>
    
**Task**: To develop a model that can predict the prices of used cars based on a set of parameters relative to the cars. It is a **regression task**.

**Dataset**: Training Dataset (train.csv): Contains features along with the target variable, indicating the car price.Test Dataset (test.csv): Contains the features but requires predictions for the price.

Additional Information:
    
**Evaluation**: The performance of the model is evaluated using the RMSE. This metric penalizes the outliers prices.

The dataset was generated from a deep learning model trained on the Used Car Price Prediction [Link]().

### Importing Libraries

In [170]:
df_sub=pd.read_csv("sample_submission.csv",na_values=["–", "not supported"])
df_train=pd.read_csv("train.csv",na_values=["–", "not supported"])
df_test=pd.read_csv("test.csv",na_values=["–", "not supported"])


df_old=pd.read_csv("used_cars.csv",na_values=["–", "not supported"])

In [171]:
df_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [172]:
df_old[['milage', 'price']] = df_old[['milage', 'price']].map(lambda x: int(''.join(re.findall(r'\d+', x))))
df_old[['milage', 'price']] = df_old[['milage', 'price']].astype(int)

In [173]:
df_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   int64 
 4   fuel_type     3792 non-null   object
 5   engine        3964 non-null   object
 6   transmission  4005 non-null   object
 7   ext_col       3994 non-null   object
 8   int_col       3876 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   int64 
dtypes: int64(3), object(9)
memory usage: 376.0+ KB


In [174]:
print("Shape Train: {}\nShape Test: {}\nShape Old: {}\n".format(df_train.shape,df_test.shape,df_old.shape))

Shape Train: (188533, 13)
Shape Test: (125690, 12)
Shape Old: (4009, 12)



In [175]:
re_ = ['model','engine','transmission','ext_col','int_col']
t=100
col='model'
df_train.loc[df_train[col].value_counts(dropna=False)[df_train[col]].values < t, col]

,model
1,LS V8
2,Silverado 2500 LT
3,G90 5.0 Ultimate
5,A6 2.0T Sport
6,A8 L 3.0T
...,...
188516,M37 x
188517,Navigator L Select
188518,S-Class S 450 4MATIC
188521,M240 i xDrive


In [176]:
df_train.brand.unique()

array(['MINI', 'Lincoln', 'Chevrolet', 'Genesis', 'Mercedes-Benz', 'Audi',
       'Ford', 'BMW', 'Tesla', 'Cadillac', 'Land', 'GMC', 'Toyota',
       'Hyundai', 'Volvo', 'Volkswagen', 'Buick', 'Rivian', 'RAM',
       'Hummer', 'Alfa', 'INFINITI', 'Jeep', 'Porsche', 'McLaren',
       'Honda', 'Lexus', 'Dodge', 'Nissan', 'Jaguar', 'Acura', 'Kia',
       'Mitsubishi', 'Rolls-Royce', 'Maserati', 'Pontiac', 'Saturn',
       'Bentley', 'Mazda', 'Subaru', 'Ferrari', 'Aston', 'Lamborghini',
       'Chrysler', 'Lucid', 'Lotus', 'Scion', 'smart', 'Karma',
       'Plymouth', 'Suzuki', 'FIAT', 'Saab', 'Bugatti', 'Mercury',
       'Polestar', 'Maybach'], dtype=object)

In [177]:
df_train[df_train["brand"]=="Ferrari"]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
251,251,Ferrari,911 GT2 RS,2022,19250,Gasoline,710.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Orange,Red,None reported,Yes,124900
433,433,Ferrari,F12berlinetta Base,2017,25520,Gasoline,6.3L V12 48V GDI DOHC,7-Speed Automatic with Auto-Shift,Gray,BLACK,None reported,NaN,449995
1554,1554,Ferrari,F12berlinetta Base,2018,28220,Gasoline,759.0HP 6.5L 12 Cylinder Engine Gasoline Fuel,8-Speed A/T,Gray,White,None reported,Yes,37400
1784,1784,Ferrari,812 Superfast Base,2018,7960,Gasoline,788.0HP 6.5L 12 Cylinder Engine Gasoline Fuel,8-Speed A/T,Red,Beige,None reported,Yes,125900
3146,3146,Ferrari,488 GTB Base,2018,17761,Gasoline,4.0L V8 32V GDI DOHC Twin Turbo,7-Speed Automatic with Auto-Shift,Rosso Corsa,Black,None reported,NaN,229900
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184666,184666,Ferrari,Roma Base,2021,3000,Gasoline,611.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Gray,Brown,None reported,Yes,215000
184690,184690,Ferrari,Roma Base,2020,19500,Gasoline,611.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Green,NaN,None reported,Yes,89000
185350,185350,Ferrari,M6 Base,2015,25500,Gasoline,560.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,None reported,Yes,51950
185697,185697,Ferrari,488 Spider Base,2016,117000,Gasoline,660.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,None reported,Yes,21000


**There is an error in the Ferrari data. The 911 GT2 is a Porsche Model and not a Ferrari. For each model there can be only one corresonding automaker.**

In [178]:
df_concat = pd.concat([df_train, df_test], axis=0)

df_concat.groupby("model")["brand"].value_counts()

model                  brand     
124 Spider Abarth      FIAT           9
                       BMW            1
128 i                  BMW           27
                       FIAT           1
                       Chrysler       1
                                     ..
tC Base                Volkswagen     1
tC Release Series 6.0  Scion         32
                       Toyota         2
xB Base                Scion         15
                       BMW            1
Name: count, Length: 4413, dtype: int64

In [179]:
df_old.groupby("model")["brand"].value_counts()

,,count
model,brand,
124 Spider Abarth,FIAT,1
128 i,BMW,1
135 i,BMW,3
135 is,BMW,1
1500 Big Horn,RAM,11
...,...,...
i8 Base,BMW,5
tC Anniversary Edition,Scion,2
tC Base,Scion,1


In [180]:
# Group by 'model' and count the occurrences of each 'brand'
brand_counts = df_concat.groupby('model')['brand'].value_counts().rename('count').reset_index()

# Find the most frequent brand for each model
most_frequent_brand = brand_counts.sort_values(['model', 'count'], ascending=False).groupby('model').head(1)

# Create a dictionary mapping 'model' to the most frequent 'brand'
brand_map = most_frequent_brand.set_index('model')['brand'].to_dict()

# Replace the 'brand' in the original DataFrame based on the 'model' and the brand_map
df_train['brand'] = df_train['model'].map(brand_map).values
df_test['brand'] = df_test['model'].map(brand_map).values

In [181]:
df_train.groupby("model")["brand"].value_counts()

,,count
model,brand,
124 Spider Abarth,FIAT,5
128 i,BMW,21
135 i,BMW,122
135 is,BMW,14
1500 Big Horn,RAM,660
...,...,...
i8 Base,BMW,60
tC Anniversary Edition,Scion,38
tC Base,Scion,15


In [182]:
df_train[df_train.fuel_type=="not supported"]#.isna().sum()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price


In [183]:
df_train.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5879
engine,925
transmission,87
ext_col,366
int_col,4527


In [184]:
df_test.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3935
engine,617
transmission,40
ext_col,223
int_col,2953


## EDA:

### **FUNCTIONS**

In [185]:
def extract_fuel_type(engine_desc):
    """Extracts fuel type from engine description using regex."""

    fuel_patterns = {
        'Gasoline': r'(?i)gasoline',
        'Diesel': r'(?i)diesel',
        'Electric': r'(?i)electric',
        'E85 Flex Fuel': r'(?i)flex fuel',
        'Hybrid': r'(?i)hybrid'
    }

    for fuel_type, pattern in fuel_patterns.items():
        if re.search(pattern, engine_desc):
            return fuel_type

    return np.nan

# Function to fill missing 'engine' values based on 'brand' and 'model'
def fill_engine(row):
  """Function to fill missing 'engine' values based on 'brand' and 'model'"""

  if pd.isnull(row['engine']):
      return engine_map.get((row['brand'], row['model']))
  else:
      return row['engine']

# Function to impute missing 'transmission' values
def impute_transmission(row):
    if pd.isnull(row['transmission']):
        return mode_transmission_by_brand_model.get((row['brand'], row['model']))
    else:
        return row['transmission']


def extract_engine_features(engine_str):
    """Extracts engine features from the engine string using regex."""

    # Horsepower
    horsepower_match = re.search(r'(\d+\.?\d*)HP', engine_str)
    horsepower = float(horsepower_match.group(1)) if horsepower_match else None

    # Engine Capacity
    capacity_match = re.search(r'(\d+\.?\d*)L', engine_str)
    engine_capacity = float(capacity_match.group(1)) if capacity_match else None

    # Engine Capacity (Modified)
    capacity_match_mod = re.search(r'(\d+\.?\d*)(?: |-)L(?:iter)?', engine_str)
    engine_capacity_mod = float(capacity_match_mod.group(1)) if capacity_match_mod else engine_capacity

    # Turbo
    turbo = 'Turbo' if 'turbo' in engine_str.lower() else 'No Turbo'

    return pd.Series({'horsepower': horsepower, 'engine_capacity': engine_capacity_mod, 'turbo': turbo})

def extract_num_cylinders(engine_str):
    """Extracts the number of cylinders from an engine string."""

    cylinder_match = re.search(r'(H|I|V|W)?(\d+)(?: |-)Cylinder|(H|I|V|W)(\d+)', engine_str)
    if cylinder_match:
        if cylinder_match.group(2) is not None:  # I, V, or W configuration with explicit "Cylinder"
            return int(cylinder_match.group(2))
        else:  # V or W configuration without "Cylinder"
            return int(cylinder_match.group(4))
    else:
        return None

# Function 1 to impute missing 'horsepower' values
def impute_horsepower_step_1(row):
    if pd.isnull(row['horsepower']):
        return median_horsepower_by_brand_model.get((row['brand'], row['model']))
    else:
        return row['horsepower']

# Function 2 to impute missing 'horsepower' values
def impute_horsepower_step_2(row):
    if pd.isnull(row['horsepower']):
        return median_horsepower_by_brand.get((row['brand']))
    else:
        return row['horsepower']

# Function 3 to impute missing 'horsepower' values
def impute_horsepower_step_3(row):
    if pd.isnull(row['horsepower']):
      if row['num_cylinders'] == 8:
        return 600
      elif row['num_cylinders'] == 12:
        return 850
      elif row['num_cylinders'] == 16:
        return 1000
    else:
        return row['horsepower']


# Function to impute missing 'transmission' values
def impute_capacity_1(row):
    if pd.isnull(row['engine_capacity']):
        return median_capacity_by_brand_model.get((row['brand'], row['model']))
    else:
        return row['engine_capacity']

# Function to impute missing 'capacity' values
def impute_capacity_2(row):
    if pd.isnull(row['engine_capacity']):
        return median_capacity_by_brand.get(row['brand'])
    else:
        return row['engine_capacity']

# Function to impute missing 'cylinder' values
def impute_cyl_1(row):
    if pd.isnull(row['num_cylinders']):
        return median_num_cylinders_by_brand_model.get((row['brand'], row['model']))
    else:
        return row['num_cylinders']

# Function to impute missing 'capacity' values
def impute_cyl_2(row):
    if pd.isnull(row['num_cylinders']):
        return median_num_cylinders_by_brand.get(row['brand'])
    else:
        return row['num_cylinders']

### **Replacing NaNs: Engine**

There are 925 missing engine entries. These are replaced with the most frequent combination of Brand and Model engine.

In [186]:
# Group by 'brand' and 'model', count occurrences of 'engine', and find the most frequent
engine_counts = df_train.groupby(['brand', 'model'])['engine'].value_counts().rename('count').reset_index()
most_frequent_engine = engine_counts.sort_values(['brand', 'model', 'count'], ascending=False).groupby(['brand', 'model']).head(1)

# Create a dictionary mapping ('brand', 'model') tuples to the most frequent 'engine'
engine_map = most_frequent_engine.set_index(['brand', 'model'])['engine'].to_dict()

In [187]:
# Apply the function to fill missing values in 'engine'
df_train['engine'] = df_train.apply(fill_engine, axis=1)
df_test['engine'] = df_test.apply(fill_engine, axis=1)

# Apply the function to fill missing values in 'engine'
df_train['engine'] = df_train.apply(fill_engine, axis=1)
df_test['engine'] = df_test.apply(fill_engine, axis=1)

# Apply the function fillna'
df_train['engine'] = df_train['engine'].fillna("Missing", axis=0)
df_test['engine'] = df_test['engine'].fillna("Missing", axis=0)

# Print the count of missing values in 'engine'
print(f"Missing values in 'engine' after filling: {df_train['engine'].isnull().sum()}")
print(f"Missing values in 'engine' after filling: {df_test['engine'].isnull().sum()}")

Missing values in 'engine' after filling: 0
Missing values in 'engine' after filling: 0


In [188]:
df_train[df_train['engine'].isna()==True]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price


In [189]:
df_test[df_test['engine'].isna()==True]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title


In [190]:
df_old[df_old['model']=="Capri XR2"]

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
1805,Mercury,Capri XR2,1993,120644,NaN,NaN,5-Speed M/T,Green,Gray,None reported,Yes,4200


### **Replacing NaNs: Fuel Type**

The type of fuel is usually embed into the engine specification, for instance "48.0HP 2.0L 4 Cylinder Engine Gasoline Fuel".

This information is extracted for the engine feature where available.

In [191]:
df_train.fuel_type.unique()

array(['Gasoline', 'E85 Flex Fuel', nan, 'Hybrid', 'Diesel',
       'Plug-In Hybrid'], dtype=object)

In [192]:
df_test.fuel_type.unique()

array(['Gasoline', 'Hybrid', 'Diesel', 'E85 Flex Fuel', nan,
       'Plug-In Hybrid'], dtype=object)

In [193]:
# Apply the function to create a new column
df_train['extracted_fuel'] = df_train['engine'].astype(str).apply(extract_fuel_type)
df_test['extracted_fuel'] = df_test['engine'].astype(str).apply(extract_fuel_type)

# Fill missing values in 'fuel_type' with values from 'extracted_fuel'
df_train['fuel_type'] = df_train['fuel_type'].fillna(df_train['extracted_fuel'])
df_test['fuel_type'] = df_test['fuel_type'].fillna(df_test['extracted_fuel'])

# Drop the temporary column
df_train.drop('extracted_fuel', axis=1, inplace=True)
df_test.drop('extracted_fuel', axis=1, inplace=True)

# If no info is available, the entry is populated with "Gasoline" 'extracted_fuel'
df_train['fuel_type'] = df_train['fuel_type'].fillna("Gasoline")
df_test['fuel_type'] = df_test['fuel_type'].fillna("Gasoline")

# Print the count of missing values in 'fuel_type'
print(f"Missing values in 'fuel_type' after filling: {df_train['fuel_type'].isnull().sum()}")
print(f"Missing values in 'fuel_type' after filling: {df_test['fuel_type'].isnull().sum()}")

Missing values in 'fuel_type' after filling: 0
Missing values in 'fuel_type' after filling: 0


In [194]:
df_train.fuel_type.unique()

array(['Gasoline', 'E85 Flex Fuel', 'Electric', 'Hybrid', 'Diesel',
       'Plug-In Hybrid'], dtype=object)

In [195]:
# Filter rows where brand is 'Tesla' and fuel_type is 'Gasoline'
tesla_gasoline_rows = df_train[(df_train['brand'] == 'Tesla') & (df_train['fuel_type'] == 'Gasoline')]
tesla_gasoline_rows_test = df_test[(df_test['brand'] == 'Tesla') & (df_test['fuel_type'] == 'Gasoline')]

# Update 'fuel_type' to 'Electric' for these rows
df_train.loc[tesla_gasoline_rows.index, 'fuel_type'] = 'Electric'
df_test.loc[tesla_gasoline_rows_test.index, 'fuel_type'] = 'Electric'

# Filter again to check if any 'Tesla' cars still have 'Gasoline' as fuel_type
remaining_tesla_gasoline_rows = df_train[(df_train['brand'] == 'Tesla') & (df_train['fuel_type'] == 'Gasoline')]
remaining_tesla_gasoline_rows_test = df_test[(df_test['brand'] == 'Tesla') & (df_test['fuel_type'] == 'Gasoline')]

# Print the number of remaining rows
print(f"Number of Tesla cars with 'Gasoline' as fuel_type after correction - Train: {len(remaining_tesla_gasoline_rows)}")
print(f"Number of Tesla cars with 'Gasoline' as fuel_type after correction - Train: {len(remaining_tesla_gasoline_rows_test)}")

Number of Tesla cars with 'Gasoline' as fuel_type after correction - Train: 0
Number of Tesla cars with 'Gasoline' as fuel_type after correction - Train: 0


**Conclusion**: total number of missing values has been reduced from 5879 to 951.

### **Replacing NaN: Transmission"**

In [196]:
# Group by 'brand' and 'model', compute the mode of 'transmission'
mode_transmission_by_brand_model = df_train.groupby(['brand', 'model'])['transmission'].agg( lambda x: pd.Series.mode(x)[0] ).to_dict()

# Apply the function to fill missing values
df_train['transmission'] = df_train.apply(impute_transmission, axis=1)
df_test['transmission'] = df_test.apply(impute_transmission, axis=1)

# Count the number of missing values in 'transmission' after imputation
missing_transmission_after = df_train['transmission'].isnull().sum()
missing_transmission_after_test = df_test['transmission'].isnull().sum()

print(f"Missing values in 'transmission' after filling (train): {missing_transmission_after}")
print(f"Missing values in 'transmission' after filling (test): {missing_transmission_after_test}")

Missing values in 'transmission' after filling (train): 0
Missing values in 'transmission' after filling (test): 0


In [197]:
df_total = pd.concat([df_train, df_test])
df_total.transmission.unique()
{a:"" for a in df_total.transmission.unique()}

df_total[df_total.transmission=="SCHEDULED FOR OR IN PRODUCTION"]#.head(3) #ESTRACT TURBO

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
15137,15137,BMW,M440 i xDrive,2021,61145,Hybrid,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Alpine White,Black,None reported,NaN,83598.0
24481,24481,Audi,Q5 2.0T Premium Plus,2019,36486,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,SCHEDULED FOR OR IN PRODUCTION,Gray,Red,None reported,Yes,23750.0
74979,74979,BMW,M240 i,2021,13177,Gasoline,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Brilliant Silver Metallic,Black,None reported,NaN,72598.0
76188,76188,BMW,M3 CS,2022,26609,Gasoline,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Mythos Black Metallic,White,None reported,NaN,35645.0
115301,115301,BMW,X6 xDrive40i,2020,28567,Gasoline,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Carbon Black Metallic,Black,At least 1 accident or damage reported,NaN,54998.0
136829,136829,BMW,530 i xDrive,2019,48156,Hybrid,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Glacier Silver Metallic,Canberra Beige/Black,None reported,NaN,31698.0
35202,223735,BMW,X3 xDrive30i,2021,2492,Gasoline,2.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Black Sapphire Metallic,Canberra Beige,None reported,NaN,NaN
41876,230409,BMW,840 Gran Coupe i xDrive,2020,23647,Gasoline,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Black Sapphire Metallic,Black,None reported,NaN,NaN
42696,231229,Mercedes-Benz,GLE 350 Base 4MATIC,2022,94101,Gasoline,2.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Black,Silk Beige/Espresso Brown,None reported,NaN,NaN
48605,237138,Audi,Q8 3.0T Prestige,2019,25348,Gasoline,349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,SCHEDULED FOR OR IN PRODUCTION,Black Sapphire Metallic,Black,At least 1 accident or damage reported,Yes,NaN


In [198]:
# Create a color mapping dictionary (You'll likely need to expand this)
trans_map = {'A/T': 'A/T',
              'Transmission w/Dual Shift Mode': 'Dual Shift Mode',
              '7-Speed A/T': '7-Speed A/T',
              '8-Speed A/T': '8-Speed A/T',
              '10-Speed Automatic': '10-Speed A/T',
              '1-Speed A/T': '1-Speed A/T',
              '6-Speed A/T': '6-Speed A/T',
              '10-Speed A/T': '10-Speed A/T',
              '9-Speed A/T': '9-Speed A/T',
              '8-Speed Automatic': '8-Speed A/T',
              '9-Speed Automatic': '9-Speed A/T',
              '5-Speed A/T': '5-Speed A/T',
              'Automatic': 'A/T',
              '7-Speed Automatic with Auto-Shift': '7-Speed A/T',
              'CVT Transmission': 'CVT',
              '5-Speed M/T': '5-Speed M/T',
              'M/T': 'M/T',
              '6-Speed M/T': '6-Speed M/T',
              '6-Speed Automatic': '6-Speed A/T',
              '4-Speed Automatic': '4-Speed A/T',
              '7-Speed M/T': 'M/T',
              '2-Speed A/T': '1-Speed A/T',
              '1-Speed Automatic': '1-Speed A/T',
              'Automatic CVT': 'CVT',
              '4-Speed A/T': '4-Speed A/T',
              '6-Speed Manual': '6-Speed M/T',
              'Transmission Overdrive Switch': 'A/T',
              '8-Speed Automatic with Auto-Shift': '8-Speed A/T',
              '7-Speed Manual': 'M/T',
              '7-Speed Automatic': '7-Speed A/T',
              '9-Speed Automatic with Auto-Shift': '9-Speed A/T',
              '6-Speed Automatic with Auto-Shift': '6-Speed A/T',
              '6-Speed Electronically Controlled Automatic with O': '6-Speed A/T',
              'F': 'M/T',
              'CVT-F': 'CVT',
              '8-Speed Manual': 'M/T',
              'Manual': 'M/T',
              '2': '1-Speed A/T',
              '6 Speed At/Mt': '6-Speed A/T',
              '5-Speed Automatic': '5-Speed A/T',
              '2-Speed Automatic': '1-Speed A/T',
              '8-SPEED A/T': '8-Speed A/T',
              '7-Speed': 'M/T',
              'Variable': 'CVT',
              'Single-Speed Fixed Gear': '1-Speed A/T',
              '8-SPEED AT': '8-Speed A/T',
              '10-Speed Automatic with Overdrive': '10-Speed A/T',
              '7-Speed DCT Automatic': '7-Speed A/T',
              'SCHEDULED FOR OR IN PRODUCTION': 'A/T',
              '6-Speed': '6-Speed M/T',
              '6 Speed Mt': '6-Speed M/T'}

# Function to map exotic gear to base gearbox
def map_to_base_gear(gear):
    return trans_map.get(gear, gear)  # If not found in map, keep the original color

# Apply the mapping to 'ext_col'
df_train['transmission'] = df_train['transmission'].astype(str).apply(map_to_base_gear)
df_test['transmission'] = df_test['transmission'].astype(str).apply(map_to_base_gear)

# Display the updated unique values in 'ext_col'
print("\nUnique values in 'transmission' after mapping - Train:\n", df_test['transmission'].unique())
print("\nUnique values in 'transmission' after mapping - Test:\n", df_train['transmission'].unique())


Unique values in 'transmission' after mapping - Train:
 ['6-Speed A/T' '8-Speed A/T' '10-Speed A/T' 'A/T' 'Dual Shift Mode'
 '7-Speed A/T' '5-Speed M/T' '9-Speed A/T' '5-Speed A/T' 'CVT'
 '6-Speed M/T' 'M/T' '1-Speed A/T' '4-Speed A/T']

Unique values in 'transmission' after mapping - Test:
 ['A/T' 'Dual Shift Mode' '7-Speed A/T' '8-Speed A/T' '10-Speed A/T'
 '1-Speed A/T' '6-Speed A/T' '9-Speed A/T' '5-Speed A/T' 'CVT'
 '5-Speed M/T' 'M/T' '6-Speed M/T' '4-Speed A/T']


In [199]:
set(df_train['transmission'].unique()) - set(df_test['transmission'].unique())

set()

In [200]:
#plot_categorical_columns(df_train)

### **Replacing NaN: ext_col**

In [201]:
#df_train[df_train.ext_col.isna()==True]

In [202]:
df_train[(df_train.ext_col=="Custom")] #&(df_train.brand=="Ferrari")

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price


In [203]:
# Create a color mapping dictionary (You'll likely need to expand this)
color_map = {
    'Soul Red Crystal Metallic': 'Red',
    'nan':np.nan,
    'Ruby Red Metallic Tinted Clearcoat': 'Red',
    'Rosso Corsa': 'Red',
    'verde': 'Green',
    'Snowflake White Pearl Metallic': 'White',
    'Gray' : 'Grey',
    'Santorini Black Metallic' : 'Black',
    'Ebony Twilight Metallic' : 'Black',
    'Magnetite Black Metallic' : 'Black',
    'Diamond Black' : 'Black',
    'Vega Blue' : 'Blue',
    'Beige': 'Beige',
     'Gold': 'Yellow',
     'Platinum White Pearl': 'White',
     'Metallic': 'Grey',
     'White Frost Tri-Coat': 'White',
     'Firecracker Red Clearcoat': 'Red',
     'Phytonic Blue Metallic': 'Blue',
     'Blu': 'Blue',
     'Orange': 'Orange',
     'Brown': 'Beige',
     'Brilliant Silver Metallic': 'Grey',
     'Black Raven': 'Black',
     'Black Clearcoat': 'Black',
     'Firenze Red': 'Red',
     'Agate Black Metallic': 'Black',
     'Glacial White Pearl': 'White',
     'Majestic Plum Metallic': 'Custom',
     'designo Diamond White Metallic': 'White',
     'Oxford White': 'White',
     'Black Sapphire Metallic': 'Black',
     'Mythos Black': 'Black',
     'Granite Crystal Clearcoat Metallic': 'Grey',
     'White Diamond Tri-Coat': 'White',
     'Magnetite Gray Metallic': 'Grey',
     'Carpathian Grey Premium Metallic': 'Grey',
     'designo Diamond White Bright': 'White',
     'Phantom Black Pearl Effect / Black Roof': 'Black',
     'Nebula Gray Pearl': 'Grey',
     'Deep Crystal Blue Mica': 'Blue',
     'Flame Red Clearcoat': 'Red',
     'Lunar Blue Metallic': 'Blue',
     'Bright White Clearcoat': 'White',
     'Rapid Red Metallic Tinted Clearcoat': 'Red',
     'Caviar': 'Red',
     'Dark Ash Metallic': 'Grey',
     'Velvet Red Pearlcoat': 'Red',
     'Silver Zynith': 'Grey',
     'Super Black': 'Black',
     'Antimatter Blue Metallic': 'Blue',
     'Dark Moon Blue Metallic': 'Blue',
     'Summit White': 'White',
     'Ebony Black': 'Black',
     'Black Cherry': 'Custom',
     'Delmonico Red Pearlcoat': 'Red',
     'Platinum Quartz Metallic': 'White',
     'Ultra White': 'White',
     'Python Green': 'Green',
     'Garnet Red Metallic': 'Red',
     'Snow White Pearl': 'White',
     'Cajun Red Tintcoat': 'Red',
     'Midnight Black Metallic': 'Black',
     'Diamond White': 'White',
     'Mythos Black Metallic': 'Black',
     'Soul Red Crystal Metallic': 'Red',
     'Atomic Silver': 'Grey',
     'Obsidian': 'Black',
     'Magnetic Metallic': 'Grey',
     'Twilight Blue Metallic': 'Blue',
     'Star White': 'White',
     'Stormy Sea': 'Blue',
     'Tango Red Metallic': 'Red',
     'Hyper Red': 'Red',
     'Portofino Gray': 'Grey',
     'MANUFAKTUR Diamond White Bright': 'White',
     'Snowflake White Pearl': 'White',
     'Patriot Blue Pearlcoat': 'Blue',
     'Tungsten Metallic': 'Grey',
     'Chronos Gray Metallic': 'Grey',
     'Silver Ice Metallic': 'Grey',
     'Daytona Gray Pearl Effect': 'Grey',
     'Ruby Red Metallic Tinted Clearcoat': 'Red',
     'Alpine White': 'White',
     'Eminent White Pearl': 'White',
     'Manhattan Noir Metallic': 'Black',
     'Quicksilver Metallic': 'Grey',
     'Stellar Black Metallic': 'Black',
     'Sparkling Silver': 'Grey',
     'Blueprint': 'Blue',
     'Crystal Black Silica': 'Black',
     'Black Noir Pearl': 'Black',
     'Arancio Borealis': 'Orange',
     'Typhoon Gray': 'Grey',
     'Ibis White': 'White',
     'Graphite Grey': 'Grey',
     'Mineral White': 'White',
     'Midnight Black': 'Black',
     'Northsky Blue Metallic': 'Blue',
     'Alta White': 'White',
     'Brilliant Black': 'Black',
     'Jet Black Mica': 'Black',
     'Daytona Gray Pearl Effect w/ Black Roof': 'Grey',
     'Redline Red': 'Red',
     'Glacier Silver Metallic': 'Grey',
     'Magnetic Black': 'Black',
     'Chronos Gray': 'Grey',
     'Red Quartz Tintcoat': 'Red',
     'Nero Noctis': 'Black',
     'Firenze Red Metallic': 'Red',
     'Iridescent Pearl Tricoat': 'White',
     'Twilight Black': 'Black',
     'Radiant Red Metallic II': 'Red',
     'Blue Metallic': 'Blue',
     'Glacier White': 'White',
     'Daytona Gray': 'Grey',
     'Rosso Mars Metallic': 'Red',
     'Wolf Gray': 'Grey',
     'Santorin Black': 'Black',
     'Designo Magno Matte': 'Grey',
     'Emerald Green Metallic': 'Green',
     'Ruby Flare Pearl': 'Red',
     'Lunar Silver Metallic': 'Grey',
     'Eiger Grey Metallic': 'Grey',
     'Quartzite Grey Metallic': 'Grey',
     'Barcelona Red': 'Red',
     'Beluga Black': 'Black',
     'Matador Red Metallic': 'Red',
     'Billet Silver Metallic Clearcoat': 'Grey',
     'Anodized Blue Metallic': 'Blue',
     'Black Forest Green': 'Green',
     'Ice Silver Metallic': 'Grey',
     'Sandstone Metallic': 'Beige',
     'Magnetic Gray Clearcoat': 'Grey',
     'Crystal Black Pearl': 'Black',
     'Pacific Blue Metallic': 'Blue',
     'Stone Gray Metallic': 'Grey',
     'Iconic Silver Metallic': 'Grey',
     'Dark Sapphire': 'Blue',
     'Onyx': 'Black',
     'Aventurine Green Metallic': 'Green',
     'China Blue': 'Blue',
     'Majestic Black Pearl': 'Black',
     'Midnight Silver Metallic': 'Grey',
     'Sting Gray Clearcoat': 'Grey',
     'Glacier Blue Metallic': 'Blue',
     'BLACK': 'Black',
     'Chalk': 'White',
     'Dark Matter Metallic': 'Black',
     'Infrared Tintcoat': 'Red',
     'Iridium Metallic': 'Grey',
     'Fuji White': 'White',
     'Alfa White': 'White',
     'Kodiak Brown Metallic': 'Beige',
     'Aurora Black': 'Black',
     'Onyx Black': 'Black',
     'Nightfall Gray Metallic': 'Grey',
     'Obsidian Black Metallic': 'Black',
     'Phantom Black': 'Black',
     'Remington Red Metallic': 'Red',
     'designo Diamond White': 'White',
     'Lizard Green': 'Green',
     'Rosso Corsa': 'Red',
     'Shadow Gray Metallic': 'Grey',
     'Florett Silver': 'Grey',
     'Quartz White': 'White',
     'DB Black Clearcoat': 'Black',
     'Yulong White': 'White',
     'Eiger Grey': 'Grey',
     'Custom Color': 'Custom',
     'Electric Blue Metallic': 'Blue',
     'Tempest': 'Blue',
     'Lunar Rock': 'Grey',
     'Mosaic Black Metallic': 'Black',
     'Gecko Pearlcoat': 'Green',
     'White Clearcoat': 'White',
     'BLU ELEOS': 'Blue',
     'Granite Crystal Metallic Clearcoat': 'Grey',
     'Rich Garnet Metallic': 'Red',
     'Graphite Grey Metallic': 'Grey',
     'Bianco Icarus Metallic': 'White',
     'Satin Steel Metallic': 'Grey',
     'BLUE': 'Blue',
     'Moonlight Cloud': 'Blue',
     'Matador Red Mica': 'Red',
     'Emin White': 'White',
     'Machine Gray Metallic': 'Grey',
     'White Platinum Tri-Coat Metallic': 'White',
     'Cobra Beige Metallic': 'Beige',
     'Cayenne Red Tintcoat': 'Red',
     'Shoreline Blue Pearl': 'Blue',
     'Vik Black': 'Black',
     'Shimmering Silver': 'Grey',
     'Bianco Monocerus': 'White',
     'Carbonized Gray Metallic': 'Grey',
     'Carrara White Metallic': 'White',
     'Dark Slate Metallic': 'Black',
     'Dark Graphite Metallic': 'Black',
     'Sonic Silver Metallic': 'Grey',
     'White Knuckle Clearcoat': 'White',
     'Titanium Silver': 'Grey',
     'Anthracite Blue Metallic': 'Blue',
     'Black Obsidian': 'Black',
     'Polymetal Gray Metallic': 'Grey',
     'Orca Black Metallic': 'Black',
     'Wind Chill Pearl': 'White',
     'Blue Reflex Mica': 'Blue',
     'Dark Moss': 'Green',
     'Selenite Grey Metallic': 'Grey',
     'Kemora Gray Metallic': 'Grey',
     'Nightfall Mica': 'Blue',
     'Liquid Platinum': 'Grey',
     'Mountain Air Metallic': 'Blue',
     'Kinetic Blue': 'Blue',
     'Santorini Black': 'Black',
     'Carbon Black Metallic': 'Black',
     'Gentian Blue Metallic': 'Blue',
     'Red Multi': 'Red',
     'Super White': 'White',
     'Pearl White': 'White',
     'Typhoon Gray Metallic': 'Grey',
     'Navarra Blue Metallic': 'Blue',
     'Bianco Isis': 'White',
     'Navarra Blue': 'Blue',
     'Volcano Grey Metallic': 'Grey',
     'Arctic Gray Metallic': 'Grey',
     'Pure White': 'White',
     'Baltic Gray': 'Grey',
     'Glacier White Metallic': 'White',
     'Frozen Dark Silver Metallic': 'Grey',
     'Magnetic Gray Metallic': 'Grey',
     'Gun Metallic': 'Grey',
     'Siren Red Tintcoat': 'Red',
     'Deep Blue Metallic': 'Blue',
     'Cirrus Silver Metallic': 'Grey',
     'Deep Black Pearl Effect': 'Black',
     'Granite': 'White',
     'Sunset Drift Chromaflair': 'Orange',
     'Oryx White Prl': 'White',
     'Dark Gray Metallic': 'Grey',
     'Bayside Blue': 'Blue',
     'Pink': 'Custom',
     'Ice': 'White',
     'Mango Tango Pearlcoat': 'Orange',
     'Burnished Bronze Metallic': 'Beige',
     'Verde': 'Green',
     'Arctic White': 'White',
     'Portofino Blue Metallic': 'Blue',
     'Dazzling White': 'White',
     'Nero Daytona': 'Black',
     'Nautical Blue Pearl': 'Blue',
     'Imperial Blue Metallic': 'Blue',
     'Vulcano Black Metallic': 'Black',
     'Silver Radiance': 'Grey',
     'Hellayella Clearcoat': 'Yellow',
     'Jungle Green': 'Green',
     'C / C': 'Custom',
     'Yulong': 'White',
     'Pristine White': 'White',
     'Silky Silver': 'Grey',
     'Caspian Blue': 'Blue',
     'Sangria Red': 'Red',
     'Donington Grey Metallic': 'Grey',
     'Apex Blue': 'Blue',
     'Rift Metallic': 'Grey',
     'Fountain Blue': 'Blue',
     'Balloon White': 'White',
     'Matte White': 'White',
     'Frozen White': 'White',
     'Pacific Blue': 'Blue',
     'Rosso': 'Red',
     'Ironman Silver': 'Grey',
     'Octane Red Pearlcoat': 'Red',
     'Selenite Gray Metallic': 'Grey',
     'Hydro Blue Pearlcoat': 'Blue',
     'Ingot Silver Metallic': 'Grey',
     'Quartz Blue Pearl': 'Blue',
     'Lunare White Metallic': 'White',
     'Ember Pearlcoat': 'Black',
     'Brands Hatch Gray Metallic': 'Grey',
     'Navarre Blue': 'Blue',
     'Midnight Blue Metallic': 'Blue',
     'Shadow Black': 'Black',
     'Go Mango!': 'Orange',
     'Maximum Steel Metallic': 'Grey',
     'Silver Flare Metallic': 'Grey',
     'Billet Clearcoat Metallic': 'Grey',
     'Hampton Gray': 'Grey',
     'Red Obsession': 'Red',
     'Silver Mist': 'Grey',
     'Scarlet Ember': 'Red',
     'Crimson Red Tintcoat': 'Red',
     'Tan': 'Beige',
     'Isle of Man Green Metallic': 'Green',
     'Crystal Black': 'Black',
     'Glacier': 'White',
     'Iridium Silver Metallic': 'Grey',
     'Bronze Dune Metallic': 'Beige',
     'Maroon': 'Red',
     'Platinum Gray Metallic': 'Grey',
     'Passion Red': 'Red',
     'Silician Yellow': 'Yellow',
     'Volcanic Orange': 'Orange',
     'Crystal White Pearl': 'White',
     'Reflex Silver': 'Grey',
     'Blue Caelum': 'Blue',
     'Thunder Gray': 'Grey',
     'Ultra Black': 'Black',
     'Indus Silver': 'Grey',
     'Horizon Blue': 'Blue',
     'Grigio Nimbus': 'Grey',
     'Carpathian Grey': 'Grey',
     'Ametrin Metallic': 'Red',
     'Jupiter Red': 'Red',
     'GT SILVER': 'Grey',
    'Purple':"Custom",
    'Silver':'Grey'}


# Function to map exotic colors to base colors
def map_to_base_color(color):
    return color_map.get(color, color)  # If not found in map, keep the original color

# Apply the mapping to 'ext_col'
df_train['ext_col'] = df_train['ext_col'].astype(str).apply(map_to_base_color)
df_test['ext_col'] = df_test['ext_col'].astype(str).apply(map_to_base_color)

# Display the updated unique values in 'ext_col'
print("\nUnique values in 'ext_col' after mapping:\n", df_test['ext_col'].unique())
print("\nUnique values in 'ext_col' after mapping:\n", df_train['ext_col'].unique())


Unique values in 'ext_col' after mapping:
 ['White' 'Grey' 'Yellow' 'Black' 'Blue' 'Red' 'Green' 'Beige' 'Orange'
 'Custom' nan]

Unique values in 'ext_col' after mapping:
 ['Yellow' 'Grey' 'Blue' 'Black' 'White' 'Green' 'Custom' 'Red' 'Beige'
 'Orange' nan]


In [204]:
df_total = pd.concat([df_train, df_test])

In [205]:
# Group by 'brand' and 'model', find the most frequent 'ext_col'
mode_ext_col_by_brand_model = df_total.groupby(['brand', 'model'])['ext_col'].agg(lambda x: pd.Series.mode(x)[0]).to_dict()

# Function to fill missing 'ext_col' values
def fill_ext_col(row):
    if pd.isnull(row['ext_col']):
        return mode_ext_col_by_brand_model.get((row['brand'], row['model']))
    else:
        return row['ext_col']

# Apply the function to fill missing values
df_train['ext_col'] = df_train.apply(fill_ext_col, axis=1)
df_test['ext_col'] = df_test.apply(fill_ext_col, axis=1)

# Count the number of missing values in 'ext_col' after imputation
missing_ext_col_after = df_train['ext_col'].isnull().sum()
missing_ext_col_after_test = df_test['ext_col'].isnull().sum()

print(f"Missing values in 'ext_col' after filling: {missing_ext_col_after}")
print(f"Missing values in 'ext_col' after filling - Test: {missing_ext_col_after_test}")

Missing values in 'ext_col' after filling: 0
Missing values in 'ext_col' after filling - Test: 0


### **Replacing NaN: int_col**

In [206]:
df_total = pd.concat([df_train, df_test])
#df_total.int_col.unique()
#{a:"" for a in df_total.int_col.unique()}
#df_train[df_train.ext_col.isna()==True]

In [207]:
#df_train[(df_train.ext_col=="Custom")] #&(df_train.brand=="Ferrari")

In [208]:
color_map_int = {'Gray': 'Gray',
              'Beige': 'Beige',
              'nan':np.nan,
              'Black': 'Black',
              'Blue': 'Custom',
              'White': 'White',
              'Red': 'Red',
              'Brown': 'Beige',
              'Dark Galvanized': 'Black',
              'Parchment.': 'White',
              'Boulder': 'Gray',
              'Orange': 'Custom',
              'Medium Earth Gray': 'Gray',
              'Ebony': 'Black',
              'Canberra Beige': 'Beige',
              'Jet Black': 'Black',
              'Silver': 'Gray',
              'Light Platinum / Jet Black': 'Double',
              'Macchiato/Magmagrey': 'Double',
              'Gold': 'Custom',
              'Cloud': 'Custom',
              'Rioja Red': 'Red',
              'Global Black': 'Black',
              'Green': 'Custom',
              'Medium Stone': 'Beige',
              'Navy Pier': 'Custom',
              'Dark Ash': 'Gray',
              'BLACK': 'Black',
              'Portland': 'Beige',
              'Sandstone': 'Beige',
              'Canberra Beige/Black': 'Double',
              'Diesel Gray / Black': 'Double',
              'Sarder Brown': 'Beige',
              'Black Onyx': 'Black',
              'White / Brown': 'Double',
              'Black/Gun Metal': 'Double',
              'Slate': 'Custom',
              'Satin Black': 'Black',
              'Macchiato Beige/Black': 'Double',
              'Charcoal': 'Black',
              'Black / Express Red': 'Double',
              'Cappuccino': 'Beige',
              'Aragon Brown': 'Beige',
              'Parchment': 'White',
              'Oyster W/Contrast': 'White',
              'Adrenaline Red': 'Red',
              'Ebony.': 'Black',
              'Shara Beige': 'Beige',
              'Graystone': 'Gray',
              'Pearl Beige': 'Beige',
              'Nero Ade': 'Black',
              'Graphite': 'Gray',
              'Tan/Ebony/Ebony': 'Black',
              'Charcoal Black': 'Black',
              'Medium Ash Gray': 'Gray',
              'Ebony Black': 'Black',
              'Light Titanium': 'Gray',
              'Sakhir Orange': 'Custom',
              'Tan': 'Beige',
              'Rock Gray': 'Gray',
              'Brandy': 'Beige',
              'Carbon Black': 'Black',
              'Amber': 'Beige',
              'Black w/Red Stitching': 'Double',
              'Hotspur': 'Custom',
              'Chateau': 'Beige',
              'Ice': 'White',
              'Deep Garnet': 'Red',
              'Blk': 'Black',
              'Grace White': 'White',
              'Oyster/Black': 'Double',
              'Mesa': 'Beige',
              'Espresso': 'Beige',
              'Black/Graphite': 'Double',
              'Ebony / Ebony Accents': 'Beige',
              'Tan/Ebony': 'Beige',
              'Ceramic': 'Red',
              'Medium Dark Slate': 'Gray',
              'Graphite w/Gun Metal': 'Gray',
              'Obsidian Black': 'Black',
              'Cocoa / Dune': 'Beige',
              'Roast': 'Beige',
              'Yellow': 'Custom',
              'Hotspur Hide': 'Custom',
              'Gray w/Blue Bolsters': 'Double',
              'Chestnut': 'Beige',
              'Saiga Beige': 'Beige',
              'ORANGE': 'Custom',
              'Charles Blue': 'Custom',
              'Walnut': 'Beige',
              'Ivory / Ebony': 'Double',
              'Caramel': 'Beige',
              'Pimento Red w/Ebony': 'Double',
              'Saddle Brown': 'Beige',
              'Dark Gray': 'Gray',
              'Silk Beige/Espresso Brown': 'Beige',
              'Black / Brown': 'Double',
              'Ebony/Light Oyster Stitch': 'Double',
              'Ebony / Pimento': 'Double',
              'Mistral Gray / Raven': 'Double',
              'Giallo Taurus / Nero Ade': 'Double',
              'Tension': 'Custom',
              'Medium Pewter': 'Gray',
              'Black / Saddle': 'Double',
              'Camel Leather': 'Beige',
              'Black/Saddle Brown': 'Double',
              'Macchiato': 'Beige',
              'Anthracite': 'Black',
              'Mocha': 'Beige',
              'Whisper Beige': 'Beige',
              'Titan Black / Quarzit': 'Double',
              'Sahara Tan': 'Beige',
              'Porpoise': 'Beige',
              'Black/Red': 'Double',
              'Titan Black': 'Black',
              'AMG Black': 'Black',
              'Deep Cypress': 'Custom',
              'Light Slate': 'Gray',
              'Red / Black': 'Double',
              'Beluga Hide': 'White',
              'Tupelo': 'Beige',
              'Gideon': 'Gray',
              'Rhapsody Blue': 'Custom',
              'Medium Light Camel': 'Beige',
              'Almond Beige': 'Beige',
              'Black / Gray': 'Double',
              'Nero': 'Black',
              'Agave Green': 'Custom',
              'Deep Chestnut': 'Beige',
              'Dark Auburn': 'Red',
              'Shale': 'Gray',
              'Silk Beige/Black': 'Double',
              'BEIGE': 'Beige',
              'Magma Red': 'Red',
              'Linen': 'Beige',
              'Black / Stone Grey': 'Double',
              'Sand Beige': 'Beige',
              'Red/Black': 'Double',
              'Bianco Polar': 'White',
              'Light Gray': 'Gray',
              'Platinum': 'Gray',
              'Sport': 'Gray',
              'Ash': 'Gray',
              'Black / Graphite': 'Double',
              'Nougat Brown': 'Beige',
              'Camel': 'Beige',
              'Mountain Brown': 'Beige',
              'Pimento / Ebony': 'Double',
              'Classic Red': 'Red',
              'Sakhir Orange/Black': 'Double',
              'Cobalt Blue': 'Custom',
              'Very Light Cashmere': 'Beige',
              'Kyalami Orange': 'Custom',
              'Orchid': 'Custom',
              'Beluga': 'White',
              'WHITE': 'White'}

In [209]:
# Function to map exotic colors to base colors
def map_to_base_color(color):
    return color_map_int.get(color, color)  # If not found in map, keep the original color

# Apply the mapping to 'ext_col'
df_train['int_col'] = df_train['int_col'].astype(str).apply(map_to_base_color)
df_test['int_col'] = df_test['int_col'].astype(str).apply(map_to_base_color)

# Display the updated unique values in 'ext_col'
print("\nUnique values in 'int_col' after mapping:\n", df_test['int_col'].unique())
print("\nUnique values in 'int_col' after mapping:\n", df_train['int_col'].unique())


Unique values in 'int_col' after mapping:
 ['Beige' 'Black' 'Gray' nan 'Red' 'Double' 'White' 'Custom']

Unique values in 'int_col' after mapping:
 ['Gray' 'Beige' 'Black' nan 'Custom' 'White' 'Red' 'Double']


In [210]:
# Display the updated unique values in 'ext_col'
print("\nUnique values in 'int_col' after mapping:\n", df_test['int_col'].value_counts())
print("\nUnique values in 'int_col' after mapping:\n", df_train['int_col'].value_counts())


Unique values in 'int_col' after mapping:
 int_col
Black     75831
Beige     21629
Gray      15102
Red        3758
White      3391
Custom     1784
Double     1242
Name: count, dtype: int64

Unique values in 'int_col' after mapping:
 int_col
Black     114009
Beige      32149
Gray       22557
Red         5560
White       5193
Custom      2792
Double      1746
Name: count, dtype: int64


In [211]:
df_total = pd.concat([df_train, df_test])

In [212]:
# Group by 'brand' and 'model', find the most frequent 'ext_col'
mode_int_col_by_brand_model = df_total.groupby(['brand', 'model'])['int_col'].agg(lambda x: pd.Series.mode(x)[0]).to_dict()

# Function to fill missing 'ext_col' values
def fill_int_col(row):
    if pd.isnull(row['int_col']):
        return mode_int_col_by_brand_model.get((row['brand'], row['model']))
    else:
        return row['int_col']

# Apply the function to fill missing values
df_train['int_col'] = df_train.apply(fill_int_col, axis=1)
df_test['int_col'] = df_test.apply(fill_int_col, axis=1)

# Count the number of missing values in 'ext_col' after imputation
missing_int_col_after = df_train['int_col'].isnull().sum()
missing_int_col_after_test = df_test['int_col'].isnull().sum()

print(f"Missing values in 'ext_col' after filling: {missing_int_col_after}")
print(f"Missing values in 'ext_col' after filling - Test: {missing_int_col_after_test}")

Missing values in 'ext_col' after filling: 0
Missing values in 'ext_col' after filling - Test: 0


### **Replacing NaN: accident**

The accident column is of type Object and thus likely contains categorical data. We will examine the non-null values to understand the categories and their frequencies. Since missing values might indicate an attempt to conceal accident history, we will impute them with a new category 'Not disclosed'.

The missing values in the 'accident' column have been successfully imputed with the new category 'Not disclosed'. This approach reflects the potential unethical behavior of concealing accident history to achieve a higher selling price. The output confirms that there are no missing values remaining in this column after the imputation process.

In [213]:
df_train['accident'].unique()

array(['None reported', 'At least 1 accident or damage reported', nan],
      dtype=object)

In [214]:
# Fill missing values in 'accident' with 'Not disclosed'
df_train['accident'] = df_train['accident'].fillna('Not disclosed')
df_test['accident'] = df_test['accident'].fillna('Not disclosed')

# Count the number of missing values in 'accident' after imputation
missing_accident_after_train = df_train['accident'].isnull().sum()
missing_accident_after_test = df_train['accident'].isnull().sum()

# Display the count of missing values
print(f"Missing values in 'accident' after filling - Train: {missing_accident_after_train}")
print(f"Missing values in 'accident' after filling - Test: {missing_accident_after_test}")

Missing values in 'accident' after filling - Train: 0
Missing values in 'accident' after filling - Test: 0


In [215]:
df_train['accident'].unique()

array(['None reported', 'At least 1 accident or damage reported',
       'Not disclosed'], dtype=object)

### **Replacing NaN: clean_title**

The `clean_title` column is of type Object and thus likely contains categorical data. We will examine the non-null values to understand the categories and their frequencies. Since missing values might indicate an attempt to conceal car's history, we will impute them with a new category 'Not disclosed'.

The missing values in the 'clean_title' column have been successfully imputed with the new category 'Not disclosed'. This approach reflects the potential unethical behavior of concealing accident history to achieve a higher selling price. The output confirms that there are no missing values remaining in this column after the imputation process.

In [216]:
df_train['clean_title'].unique()

array(['Yes', nan], dtype=object)

In [217]:
# Fill missing values in 'accident' with 'Not disclosed'
df_train['clean_title'] = df_train['clean_title'].fillna('Not disclosed')
df_test['clean_title'] = df_test['clean_title'].fillna('Not disclosed')

# Count the number of missing values in 'accident' after imputation
missing_clean_after_train = df_train['clean_title'].isnull().sum()
missing_clean_after_test = df_train['clean_title'].isnull().sum()

# Display the count of missing values
print(f"Missing values in 'clean_title' after filling - Train: {missing_clean_after_train}")
print(f"Missing values in 'clean_title' after filling - Test: {missing_clean_after_test}")

Missing values in 'clean_title' after filling - Train: 0
Missing values in 'clean_title' after filling - Test: 0


In [218]:
df_train['clean_title'].unique()

array(['Yes', 'Not disclosed'], dtype=object)

In [219]:
#plot_categorical_columns(df_train)

### EXTRACT FEATURES: engine

In [ ]:
# Apply the function and create new columns
df_train[['horsepower', 'engine_capacity', 'turbo']] = df_train['engine'].astype(str).apply(extract_engine_features)
df_test[['horsepower', 'engine_capacity', 'turbo']] = df_test['engine'].astype(str).apply(extract_engine_features)

In [ ]:
# Apply the function and create a new column
df_train['num_cylinders'] = df_train['engine'].astype(str).apply(extract_num_cylinders)
df_test['num_cylinders'] = df_test['engine'].astype(str).apply(extract_num_cylinders)

In [ ]:
df_train[df_train.num_cylinders.isnull()==True].sample(3)

#### Replacing NaN: Horsepower

In [ ]:
df_total = pd.concat([df_train, df_test])
median_horsepower_by_brand_model = df_total.groupby(['brand', 'model'])['horsepower'].median().to_dict()
median_horsepower_by_brand = df_total.groupby(['brand'])['horsepower'].median().to_dict()

In [ ]:
# Apply the function to fill missing values
df_train['horsepower'] = df_train.apply(impute_horsepower_step_1, axis=1)
df_train['horsepower'] = df_train.apply(impute_horsepower_step_2, axis=1)
df_train['horsepower'] = df_train.apply(impute_horsepower_step_3, axis=1)

df_test['horsepower'] = df_test.apply(impute_horsepower_step_1, axis=1)
df_test['horsepower'] = df_test.apply(impute_horsepower_step_2, axis=1)
df_test['horsepower'] = df_test.apply(impute_horsepower_step_3, axis=1)


# Count the number of missing values in 'horsepower' after imputation
missing_horsepower_after = df_train['horsepower'].isnull().sum()
missing_horsepower_after_test = df_test['horsepower'].isnull().sum()

# Display the count of missing values
print(f"Missing values in 'horsepower' after filling - Train: {missing_horsepower_after}")
print(f"Missing values in 'horsepower' after filling - Test: {missing_horsepower_after_test}")

In [ ]:
df_train['horsepower'].isnull().sum()

In [ ]:
#df_test[df_test.brand=="Bugatti"]

#### Replacing NaN: Capacity

In [ ]:
missing_capacity_after = df_train['engine_capacity'].isnull().sum()
missing_capacity_after_test = df_test['engine_capacity'].isnull().sum()

print(f"Missing values in 'capacity' before filling (train): {missing_capacity_after}")
print(f"Missing values in 'capacity' before filling (test): {missing_capacity_after_test}")

In [ ]:
# Group by 'brand' and 'model', compute the mode of 'engine_capacity'
median_capacity_by_brand_model = df_train.groupby(['brand', 'model'])['engine_capacity'].agg(lambda x: pd.Series.median(x) ).to_dict()
median_capacity_by_brand = df_total.groupby(['brand'])['engine_capacity'].median().to_dict()

# Apply the function to fill missing values Step 1
df_train['engine_capacity'] = df_train.apply(impute_capacity_1, axis=1)
df_test['engine_capacity'] = df_test.apply(impute_capacity_1, axis=1)

# Apply the function to fill missing values Step 2
df_train['engine_capacity'] = df_train.apply(impute_capacity_2, axis=1)
df_test['engine_capacity'] = df_test.apply(impute_capacity_2, axis=1)

# Count the number of missing values in 'transmission' after imputation
missing_capacity_after = df_train['engine_capacity'].isnull().sum()
missing_capacity_after_test = df_test['engine_capacity'].isnull().sum()

print(f"Missing values in 'capacity' after filling (train): {missing_capacity_after}")
print(f"Missing values in 'capacity' after filling (test): {missing_capacity_after_test}")

#### Replacing NaN: num_cylinders

In [ ]:
missing_cylinder_after = df_train['num_cylinders'].isnull().sum()
missing_cylinder_after_test = df_test['num_cylinders'].isnull().sum()

print(f"Missing values in 'num_cylinders' before filling (train): {missing_cylinder_after}")
print(f"Missing values in 'num_cylinders' before filling (test): {missing_cylinder_after_test}")

In [ ]:
# Group by 'brand' and 'model', compute the mode of 'num_cylinders'
median_num_cylinders_by_brand_model = df_train.groupby(['brand', 'model'])['num_cylinders'].agg(lambda x: pd.Series.median(x) ).to_dict()
median_num_cylinders_by_brand = df_total.groupby(['brand'])['num_cylinders'].median().to_dict()

# Apply the function to fill missing values Step 1
df_train['num_cylinders'] = df_train.apply(impute_cyl_1, axis=1)
df_test['num_cylinders'] = df_test.apply(impute_cyl_1, axis=1)

# Apply the function to fill missing values Step 2
df_train['num_cylinders'] = df_train.apply(impute_cyl_2, axis=1)
df_test['num_cylinders'] = df_test.apply(impute_cyl_2, axis=1)

# Count the number of missing values in 'transmission' after imputation
missing_num_cylinders_after = df_train['num_cylinders'].isnull().sum()
missing_num_cylinders_after_test = df_test['num_cylinders'].isnull().sum()

print(f"Missing values in 'capacity' after filling (train): {missing_num_cylinders_after}")
print(f"Missing values in 'capacity' after filling (test): {missing_num_cylinders_after_test}")

### Feature Engeneering

In [ ]:
def feature_engineering(df_):
    df=df_.copy()
    # Feature 1: Age of the vehicle
    df['age'] = 2025 - df['model_year']

    # Feature 2: Mileage per year
    df['milage_per_year'] = df['milage'] / df['age']

    df = df.drop(["model_year","id","engine"], axis=1)

    premium_brands=["Aston","Audi","Bentley","BMW","Cadillac","Genesis",
                    "Hummer","INFINITI","Jaguar","Land Rover","Lexus","Lotus","Lucid",
                    "Maserati","Mercedes-Benz","Polestar","Porsche","Rivian",
                    "Tesla","Volvo"]


                    # Create a new column 'premium_brand'
    df['premium_brand'] = df['brand'].apply(lambda x: 1 if x in premium_brands else 0)

    luxury_brands=["Bugatti","Ferrari","Lamborghini","Rolls-Royce","Maybach","McLaren"]

    df['luxury_brand'] = df['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

    df["engine_eff"] = (df["horsepower"]/100) * (df["num_cylinders"]/2)

    df["num_cylinders"] = df["num_cylinders"].astype("int")
    df["age"] = df["age"].astype("int")

    #transform mileange in log
    df['milage'] = np.log1p(df['milage'])

    return df

In [ ]:
df_train.head()

In [ ]:
df_train_ext = feature_engineering(df_train)
df_test_ext = feature_engineering(df_test)

In [ ]:
df_train_ext.head()

In [ ]:
df_train_ext.isnull().sum()/df_train_ext.shape[0]

In [ ]:
df_test_ext.isnull().sum()/df_test_ext.shape[0]

In [ ]:
df_train_ext.info()

In [ ]:
df_train_ext.groupby(["age","luxury_brand","premium_brand"]).agg({"price":["mean","median"]}).head(10)

## Outliers:

In [ ]:
df_train_ext.to_csv("train_extended.csv", index=False)
df_test_ext.to_csv("test_extended.csv", index=False)

#### **LUXURY BRANDS**

In [ ]:
luxury_brand_df = df_train_ext[df_train_ext['luxury_brand'] == 1]
luxury_brand_df[(luxury_brand_df.accident=="None reported")&(luxury_brand_df.price<=25_000)].shape

In [ ]:
# Filter data where `luxury_brand` is 1
luxury_brand_df = df_train_ext[df_train_ext['luxury_brand'] == 1]

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

# Histogram in the first subplot
sns.histplot(luxury_brand_df['price'], ax=axes[0], kde=False)
axes[0].set_title('Distribution of Price for Luxury Brands')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Count')

# Boxplot in the second subplot
sns.boxplot(y=luxury_brand_df['price'], ax=axes[1], hue=luxury_brand_df['accident'])
axes[1].set_title('Box Plot of Price for Luxury Brands')
axes[1].set_ylabel('Price')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
df_train_ext = df_train_ext.drop(df_train_ext[(df_train_ext.accident=="None reported")&(df_train_ext.price<=25_000)&(df_train_ext.luxury_brand==1)].index,axis=0)
df_train_ext.shape

In [ ]:
# Filter data where `luxury_brand` is 1
luxury_brand_df = df_train_ext[df_train_ext['luxury_brand'] == 1]

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

# Histogram in the first subplot
sns.histplot(luxury_brand_df['price'], ax=axes[0], kde=False)
axes[0].set_title('Distribution of Price for Luxury Brands')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Count')

# Boxplot in the second subplot
sns.boxplot(y=luxury_brand_df['price'], ax=axes[1], hue=luxury_brand_df['accident'])
axes[1].set_title('Box Plot of Price for Luxury Brands')
axes[1].set_ylabel('Price')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

#### **PREMIUM BRANDS**

In [ ]:
# Filter data where `premium` is 1
premium_brand_df = df_train_ext[df_train_ext['premium_brand'] == 1]

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

# Histogram in the first subplot
sns.histplot(premium_brand_df['price'], ax=axes[0], kde=False)
axes[0].set_title('Distribution of Price for Premium Brands')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Count')

# Boxplot in the second subplot
sns.boxplot(y=premium_brand_df['price'], ax=axes[1], hue=premium_brand_df['accident'])
axes[1].set_title('Box Plot of Price for Premium Brands')
axes[1].set_ylabel('Price')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
premium_brand_df[premium_brand_df.price>=600_000].shape

In [ ]:
premium_brand_df[(premium_brand_df.price<=4500)&(premium_brand_df.accident=="None reported")&(premium_brand_df.milage<=11)].shape

In [ ]:
df_train_ext = df_train_ext.drop(df_train_ext[(df_train_ext.accident=="None reported")&(df_train_ext.price<=4500)&(df_train_ext.premium_brand==1)&(df_train_ext.accident=="None reported")].index,axis=0)
df_train_ext = df_train_ext.drop(df_train_ext[(df_train_ext.price>=550_000)&(df_train_ext.premium_brand==1)].index,axis=0)

df_train_ext.shape

In [ ]:
# Filter data where `premium` is 1
premium_brand_df = df_train_ext[df_train_ext['premium_brand'] == 1]

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

# Histogram in the first subplot
sns.histplot(premium_brand_df['price'], ax=axes[0], kde=False)
axes[0].set_title('Distribution of Price for Premium Brands')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Count')

# Boxplot in the second subplot
sns.boxplot(y=premium_brand_df['price'], ax=axes[1], hue=premium_brand_df['accident'])
axes[1].set_title('Box Plot of Price for Premium Brands')
axes[1].set_ylabel('Price')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
#premium_brand_df.nlargest(10, "price")
#df_train_ext[df_train_ext.brand=="INFINITI"].sort_values("price", ascending=False)

#### **Popular Brands**

In [ ]:
# Filter data where `premium` is 1
poor_brand_df = df_train_ext[(df_train_ext['premium_brand'] == 0)&(df_train_ext['luxury_brand'] == 0)]

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

# Histogram in the first subplot
sns.histplot(poor_brand_df['price'], ax=axes[0], kde=False)
axes[0].set_title('Distribution of Price for Poor Brands')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Count')

# Boxplot in the second subplot
sns.boxplot(y=poor_brand_df['price'], ax=axes[1], hue=poor_brand_df['accident'])
axes[1].set_title('Box Plot of Price for Poor Brands')
axes[1].set_ylabel('Price')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
poor_brand_df[poor_brand_df.price>=350_000].sort_values("price", ascending=False).shape

In [ ]:
print(df_train_ext.shape)
df_train_ext = df_train_ext.drop(df_train_ext[(df_train_ext.price>=250_000)&(df_train_ext['premium_brand'] == 0)&(df_train_ext['luxury_brand'] == 0)].index,axis=0)
print(df_train_ext.shape)

In [ ]:
df_train_ext.reset_index(drop=True, inplace=True)

In [ ]:
# Filter data where `premium` is 1
poor_brand_df = df_train_ext[(df_train_ext['premium_brand'] == 0)&(df_train_ext['luxury_brand'] == 0)]

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

# Histogram in the first subplot
sns.histplot(poor_brand_df['price'], ax=axes[0], kde=False)
axes[0].set_title('Distribution of Price for Poor Brands')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Count')

# Boxplot in the second subplot
sns.boxplot(y=poor_brand_df['price'], ax=axes[1], hue=poor_brand_df['accident'])
axes[1].set_title('Box Plot of Price for Poor Brands')
axes[1].set_ylabel('Price')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
#df_train_ext= pd.read_csv("train_extended.csv")
#df_test_ext = pd.read_csv("test_extended.csv")
#df_train_ext.head()

## Review: Models

Remove all the models and Brands that are not in the test set:

In [ ]:
to_remove = list(set(df_train_ext.model.unique()) - set(df_test.model.unique()))
print(to_remove)

In [ ]:
for rem in to_remove:
    index_rem = df_train_ext[df_train_ext.model==rem].index
    df_train_ext = df_train_ext.drop(index_rem, axis=0)

In [ ]:
import category_encoders as ce

In [ ]:
df_train_ext.info()

In [ ]:
cat_col = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', "age", "clean_title","turbo", "num_cylinders","accident"]

# Target Encoder for XGBRegressor
train_data_encoded = df_train_ext.copy()
test_data_encoded = df_test_ext.copy()

target_encoder = ce.TargetEncoder(cols=cat_col)
train_data_encoded[cat_col] = target_encoder.fit_transform(train_data_encoded[cat_col], train_data_encoded["price"])
test_data_encoded[cat_col] = target_encoder.transform(test_data_encoded[cat_col])

In [ ]:
# # Compute the correlation matrix
correlation_matrix = train_data_encoded[train_data_encoded.select_dtypes(["float","int"]).columns].corr()
target_correlation = correlation_matrix['price']
print(target_correlation)

In [ ]:
train_data_encoded.select_dtypes("float").columns

In [ ]:
train_data_encoded.head()

In [ ]:
del train_data_encoded,correlation_matrix
gc.collect()

In [ ]:
df_train_ext[['accident', 'clean_title',"luxury_brand","premium_brand","turbo"]].describe(include ="all")

In [ ]:
df_train_ext.head(3)

In [ ]:
df_test_ext.head(3)

## Tree Models:

### **Hyperparameter Tuning- Optuna**

In [ ]:
df_train_ext.head()

In [ ]:
# Step 1: Data Preparation
y = df_train_ext['price']
X = df_train_ext.drop(columns=['price'])

use_gpu = True  # Set to False if you want to use CPU

In [ ]:
categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', "age", "clean_title", "num_cylinders","accident"]
cat_bin = [ "turbo"]
#categorical_features_2 = ['clean_title'
tot_cat = categorical_features + cat_bin

In [ ]:
# Step 2: Objective Function for Optuna
def objective(trial, model_class, X, y, use_gpu=False):

    categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', "age", "clean_title", "num_cylinders","accident"]
    cat_bin = [ "turbo"]
    tot_cat = categorical_features + cat_bin
    numeric_features = [col for col in X.columns if col not in tot_cat]

    if model_class == XGBRegressor:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 300, 600, step=5),
            'learning_rate': 0.01, #trial.suggest_loguniform('learning_rate', 0.01, 0.03),
            'max_depth': trial.suggest_int('max_depth', 5, 15),
            'min_child_weight': trial.suggest_int('min_child_weight', 6, 10),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.5),
            'subsample' :trial.suggest_float('subsample', .50, 0.95, step=0.025),
            'reg_lambda' : trial.suggest_float('reg_lambda', 1e-3, 1.0, log = True),
            'reg_alpha' :  trial.suggest_float('reg_alpha', 1e-3, 1.0, log = True),
            'random_state': 42,
            'tree_method': 'gpu_hist' if use_gpu else 'hist'
        }

        model = model_class(**params, objective='reg:squarederror')

        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', ce.TargetEncoder(), categorical_features),
                ('cat_bin', OneHotEncoder(drop="first"), cat_bin),
                ('num', 'passthrough', numeric_features)
            ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"), categorical_features),
                ('cat_bin', OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"), cat_bin),
                ('num', 'passthrough', numeric_features)
            ])
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', MinMaxScaler()),
            ('model', model)
        ])

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        rmse_scores = []

        for train_idx, val_idx in kf.split(X, y):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_val)
            rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred)))

        return np.mean(rmse_scores)

    elif model_class == LGBMRegressor:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.02, 0.03),
            'max_depth': trial.suggest_int('max_depth', 5, 30),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 8, 10),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.5),
            'random_state': 42,
            'device': 'cpu' if use_gpu else 'cpu'
        }
        model = model_class(**params, objective='regression')
        # Use CPU to avoid GPU issues
#         if use_gpu:
#             model.set_params(device='cpu')

        pipeline = Pipeline([
            ('model', model)
        ])

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        rmse_scores = []

        for train_idx, val_idx in kf.split(X, y):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_val)
            rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred)))

        return np.mean(rmse_scores)

    elif model_class == CatBoostRegressor:
        params = {
            'iterations': trial.suggest_int('iterations', 2000, 4000),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.02, 0.03),
            'depth': trial.suggest_int('depth', 5, 15),
            'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.5, 1.0),
            'random_seed': 42,
            'task_type': 'GPU' if use_gpu else 'CPU'
        }
        model = model_class(**params, loss_function='RMSE', verbose=0)

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        rmse_scores = []

        for train_idx, val_idx in kf.split(X, y):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            train_pool = Pool(X_train, y_train, cat_features=tot_cat)
            val_pool = Pool(X_val, y_val, cat_features=tot_cat)

            model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=200, verbose=0)
            y_pred = model.predict(val_pool)
            rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred)))

        return np.mean(rmse_scores)

# Step 3: Tuning Hyperparameters with Optuna
def tune_hyperparameters(X, y, model_class, n_trials, use_gpu):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    study.optimize(lambda trial: objective(trial, model_class, X, y, use_gpu=use_gpu), n_trials=n_trials)
    numeric_features = [col for col in X.columns if col not in tot_cat]
    return study  # Return the study object

In [ ]:
# Step 4: Training and Evaluation
def train_and_evaluate(model_class, params, X, y, use_gpu):
    categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', "age", "clean_title", "num_cylinders","accident"]
    cat_bin = [ "turbo"]

    tot_cat = categorical_features + cat_bin
    numeric_features = [col for col in X.columns if col not in tot_cat]

    if model_class == XGBRegressor:
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', ce.TargetEncoder(), categorical_features),
                ('cat_bin', OneHotEncoder(drop="first"), cat_bin),
                ('num', 'passthrough', numeric_features)
            ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"), categorical_features),
                ('cat_bin', OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"), cat_bin),
                ('num', 'passthrough', numeric_features)
            ])

        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', MinMaxScaler()),
            ('model', model_class(**params, objective='reg:squarederror', tree_method='gpu_hist' if use_gpu else 'auto'))
        ])
        pipeline.fit(X, y)
        y_pred = pipeline.predict(X)

    elif model_class == LGBMRegressor:
#         # Set to CPU if GPU usage is specified
#         if use_gpu:
#             params['device'] = 'cpu'

        pipeline = Pipeline([
            ('model', model_class(**params, objective='regression', verbose=-1))
        ])
        pipeline.fit(X, y)
        y_pred = pipeline.predict(X)

    elif model_class == CatBoostRegressor:
        train_pool = Pool(X, y, cat_features=tot_cat)

        # Use GPU or CPU based on the flag
        params['task_type'] = 'GPU' if use_gpu else 'CPU'

        model = model_class(**params, loss_function='RMSE', verbose=0)
        model.fit(train_pool, early_stopping_rounds=200)  # Apply early stopping
        y_pred = model.predict(train_pool)

    rmse = np.sqrt(mean_squared_error(y, y_pred))
    print(f'RMSE on train data for {model_class.__name__}: {rmse}')

    if model_class == CatBoostRegressor:
        return model  # Return the trained CatBoost model directly
    else:
        return pipeline  # Return the pipeline for other models

In [ ]:
# Step 5: Saving Best Results and Models
def save_results(study, model_class, model_name):
    best_params_file = f"{model_name}_best_params.joblib"
    joblib.dump(study.best_params, best_params_file)
    print(f"Best parameters for {model_name} saved to {best_params_file}")

    verbose_file = f"{model_name}_optuna_verbose.log"
    with open(verbose_file, "w") as f:
        f.write(str(study.trials))
    print(f"Optuna verbose for {model_name} saved to {verbose_file}")

#### XGBoost

In [ ]:
# usage with XGBRegressor
xgb_study = tune_hyperparameters(X, y, XGBRegressor, n_trials=151, use_gpu=use_gpu)
save_results(xgb_study, XGBRegressor, "XGBoost")
xgb_params = xgb_study.best_params  # Extract best parameters
xgb_model = train_and_evaluate(XGBRegressor, xgb_params, X, y, use_gpu)
#xgb_preds = xgb_model.predict(df_test)

del xgb_study, xgb_model
gc.collect()

 * Trial 130 finished with value: 72714.29728345663 and parameters:
  - 'n_estimators': 430,
  - 'learning_rate': 0.010008722503218277,
  - 'max_depth': 7,
  - 'min_child_weight': 7,
  - 'colsample_bytree': 0.4428196634007086.

* Trial 141 finished with value: 72687.0982708054 and parameters: {'n_estimators': 370, 'max_depth': 7, 'min_child_weight': 10, 'colsample_bytree': 0.41895690905135957, 'subsample': 0.75, 'reg_lambda': 0.24030084505833366, 'reg_alpha': 0.02302324434692417}. Best is trial 141 with value: 72687.0982708054.

* Trial 148 finished with value: 34140.852760465525 and parameters: {'n_estimators': 425, 'max_depth': 10, 'min_child_weight': 10, 'colsample_bytree': 0.42657449117256985, 'subsample': 0.55, 'reg_lambda': 0.051049931708831736, 'reg_alpha': 0.01446154760590466}. Best is trial 148 with value: 34140.852760465525

In [145]:
xgb_params = {'n_estimators': 370, 'max_depth': 7, 'min_child_weight': 10, 'colsample_bytree': 0.41895690905135957, 'subsample': 0.75, 'reg_lambda': 0.24030084505833366, 'reg_alpha': 0.02302324434692417}

n_splits=3
n_repeats = 3

test_results = pd.DataFrame(columns=list(range(n_splits*n_repeats)))
valid_scores = pd.DataFrame(index= list(range(n_splits*n_repeats)),columns=["Model","RMSE"])

rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats,random_state=36851234)

rskf.get_n_splits(X, X["brand"])

for i, (train_index, val_index) in enumerate(rskf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    xgb_model = train_and_evaluate(XGBRegressor, xgb_params, X_train, y_train, use_gpu)
    xgb_preds = xgb_model.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y_val, xgb_preds))
    print(f'RMSE on validation data for XGBoost: {rmse}')

    test_results[i] = xgb_model.predict(df_train_ext)
    valid_scores.loc[i, "Model"] = f"XGBoost_{i}"
    valid_scores.loc[i, "RMSE"] = rmse

gc.collect()

RMSE on train data for XGBRegressor: 50761.701597852945
RMSE on validation data for XGBoost: 78788.62086345445
RMSE on train data for XGBRegressor: 51330.35163322765
RMSE on validation data for XGBoost: 79101.40512247059
RMSE on train data for XGBRegressor: 49697.291556123455
RMSE on validation data for XGBoost: 80828.7568823819
RMSE on train data for XGBRegressor: 51033.18625570534
RMSE on validation data for XGBoost: 78934.19708406493
RMSE on train data for XGBRegressor: 49776.5748956847
RMSE on validation data for XGBoost: 79207.27505540654
RMSE on train data for XGBRegressor: 51243.72143886583
RMSE on validation data for XGBoost: 79270.74037238362
RMSE on train data for XGBRegressor: 49862.182860413945
RMSE on validation data for XGBoost: 78948.74966758444
RMSE on train data for XGBRegressor: 51133.021589318174
RMSE on validation data for XGBoost: 80294.634858692
RMSE on train data for XGBRegressor: 51895.59393530236
RMSE on validation data for XGBoost: 79720.99167283389


344

#### XGBoost Forecast

In [146]:
test_results["mean"] = test_results.mean(axis=1)
test_results.head(10)

,0,1,2,3,4,5,6,7,8,mean
0,5193.164062,6138.085938,6540.386719,9218.863281,6158.230469,3763.949219,9486.257812,7100.457031,4314.445312,6434.871094
1,1407.171875,2090.585938,8386.125000,1899.531250,5960.589844,-5050.093750,4669.320312,14115.193359,9233.542969,4745.773926
2,13089.400391,22338.292969,3659.648438,11567.363281,8714.781250,10643.226562,13667.166016,8102.441406,18080.777344,12207.011719
3,63413.867188,81766.851562,43352.519531,45272.261719,60326.296875,58712.816406,88560.921875,79215.781250,52039.125000,63628.945312
4,85247.765625,39891.328125,60700.035156,77507.554688,-2354.484375,123548.859375,123402.531250,74964.781250,41988.343750,69432.968750
5,33099.242188,31839.144531,34906.957031,35407.738281,35714.816406,30088.806641,30520.152344,36552.960938,31731.363281,33317.910156
6,27748.656250,20602.978516,28087.378906,30778.203125,25104.070312,26813.041016,30402.326172,29185.716797,28773.863281,27499.582031
7,25193.802734,23605.191406,26508.316406,20855.171875,26652.345703,26934.041016,24929.250000,23938.037109,27870.371094,25165.169922
8,34122.359375,49956.421875,37229.179688,36838.085938,30357.707031,31993.427734,40287.769531,37026.734375,30093.066406,36433.859375
9,34786.429688,26852.052734,41190.980469,32154.855469,26094.750000,24855.019531,27936.500000,29532.636719,25864.283203,29918.615234


In [147]:
df_sub["price"] = test_results["mean"]
df_sub.head()

,id,price
0,188533,6434.871094
1,188534,4745.773926
2,188535,12207.011719
3,188536,63628.945312
4,188537,69432.968750


In [148]:
df_sub.to_csv("submission_xgb_v1.csv", index=False)